In [0]:
%pip install -qqqq -U unitycatalog-ai[databricks] databricks-agents databricks-mcp
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.simplefilter(action="ignore")

In [0]:
dbutils.widgets.text(name="catalog", defaultValue="bo_cheng_dnb_demos", label="catalog")
dbutils.widgets.text(name="schema", defaultValue="agents", label="schema")
dbutils.widgets.text(name="space_id", defaultValue="", label="space_id")

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")
SPACE_ID = dbutils.widgets.get("space_id")

In [0]:
# client.delete_function(function_name=f"{CATALOG}.{SCHEMA}.genie_to_chart")

In [0]:
def genie_to_chart(genie_response_json: str, chart_type: str) -> str:
    """
    Transform Genie MCP response into a Plotly chart.
    This function handles all data extraction and transformation from Genie's nested JSON format.

    Args:
        genie_response_json (str): Raw JSON string from Genie MCP query_space tool
        chart_type (str): Type of chart - "bar", "line", or "pie"

    Returns:
        str: Plotly JSON string for direct rendering with Plotly
    """
    import json
    import plotly.express as px
    import pandas as pd

    # Parse Genie response - handle both full wrapper and extracted content
    genie_data = json.loads(genie_response_json)

    # If there's a 'content' field, parse it (double-parse scenario)
    if "content" in genie_data:
        content_data = (
            json.loads(genie_data["content"])
            if isinstance(genie_data["content"], str)
            else genie_data["content"]
        )
    else:
        # Already the content (no wrapper)
        content_data = genie_data

    # Extract columns and rows from statement_response
    columns_info = content_data["statement_response"]["manifest"]["schema"]["columns"]
    column_names = [col["name"] for col in columns_info]

    data_array = content_data["statement_response"]["result"]["data_array"]
    rows = [[value["string_value"] for value in row["values"]] for row in data_array]

    # Create DataFrame
    df = pd.DataFrame(rows, columns=column_names)

    # Convert all columns to appropriate types
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="ignore")

    # Column selection: string columns for X, last numeric for Y
    string_cols = df.select_dtypes(include=["object", "string"]).columns
    numeric_cols = df.select_dtypes(include="number").columns

    x_col = string_cols[0] if len(string_cols) > 0 else df.columns[0]
    y_col = numeric_cols[-1] if len(numeric_cols) > 0 else df.columns[-1]

    # Generate chart
    chart_functions = {
        "bar": lambda: px.bar(df, x=x_col, y=y_col, title=f"{y_col} by {x_col}"),
        "line": lambda: px.line(
            df, x=x_col, y=y_col, title=f"{y_col} by {x_col}", markers=True
        ),
        "pie": lambda: px.pie(
            df, names=x_col, values=y_col, title=f"{y_col} by {x_col}"
        ),
    }

    chart_type_key = chart_type.lower().strip()
    if chart_type_key not in chart_functions:
        raise ValueError(f"Unsupported chart type: {chart_type}")

    fig = chart_functions[chart_type_key]()

    # Return structured response with Plotly JSON

    response = {
        "plotly_json": json.loads(fig.to_json()),
        "chart_type": chart_type_key,
    }
    return json.dumps(response)


# Create the function and supply the dependency in standard PyPI format
client.create_python_function(
    func=genie_to_chart,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
    dependencies=["plotly", "pandas"],
)